# V5
* Training Data의 모든 이미지를 (28,28)픽셀로 변환해서 저장
* 학습 잘 되는지 확인

# GPU

In [1]:
import tensorflow.compat.v1 as tf1
# import tensorflow as tf2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.05
session = tf1.Session(config=config)

2022-11-05 15:30:07.556441: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 15:30:08.219192: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4053 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:b3:00.0, compute capability: 8.0


# Library

In [3]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Training 파일안의 모든 이미지 픽셀수 변경 코드

In [4]:
# 원하는 주소 안에있는 png로 끝나는 파일들 모두 (28,28)로 형태 변환
def fixel(image_path):
    img_list = os.listdir(image_path)
    img_list_jpg = [img for img in img_list if img.endswith(".png")]

    lst = []
    for i in img_list_jpg:
        img = Image.open(image_path + i)
        img = img.resize((28,28))
        img_array = np.array(img)
        lst.append(img_array)
        
    return lst

In [6]:
Tra_L = fixel("Data/Training/L/")
Tra_M = fixel("Data/Training/M/")
Tra_S = fixel("Data/Training/S/")

In [7]:
X_train = Tra_L + Tra_M + Tra_S

X_train = np.array(X_train)

X_train.shape

(7140, 28, 28, 3)

In [9]:
y_train = []

for i in ["L", "M", "S"]:
    for j in range(2380):
        y_train.append(i)
        
y_train = np.array(y_train)
y_train.shape

(7140,)

# y_val_np 문자열을 정수로 변환

In [11]:
y_train = np.where(y_train == "L", 0, y_train)
y_train = np.where(y_train == "M", 1, y_train)
y_train = np.where(y_train == "S", 2, y_train)

y_train = y_train.astype("int")

# 데이터 나누기
* 잘 학습되는구만 이제 Validation으로 정확도 높이고 Test Data로 마지막 정확도를 확인해보자

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_train, y_train,
                                                    test_size=0.2, random_state=42)

In [16]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same"),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding="same"),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
    
    
  tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

loss, acc = model.evaluate(x_test, y_test, verbose=2)

print("오차 :",loss,"정확도 :", acc)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 16)        448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 13, 13, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 11, 11, 16)        2320      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 5, 5, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 5, 5, 32)          4640      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 2, 2, 32)        